# PyTorch Useful Methods

In [1]:
import torch

### expand: copy the given tensor and concat those at desired dimension.

In [3]:
x = torch.FloatTensor([[[1, 2]],
                       [[3, 4]]])
print(x)
print(x.size())

tensor([[[1., 2.]],

        [[3., 4.]]])
torch.Size([2, 1, 2])


In [7]:
y = x.expand(*[2, 3, 2]) # == x.expand(2,3,2)
z = x.expand(2,3,2)

print(y) 
print(y.size())
print('')
print(z)
print(z.size())

tensor([[[1., 2.],
         [1., 2.],
         [1., 2.]],

        [[3., 4.],
         [3., 4.],
         [3., 4.]]])
torch.Size([2, 3, 2])

tensor([[[1., 2.],
         [1., 2.],
         [1., 2.]],

        [[3., 4.],
         [3., 4.],
         [3., 4.]]])
torch.Size([2, 3, 2])


#### Implement expand with cat.

In [8]:
y = torch.cat([x, x, x], dim=1)

print(y)
print(y.size())

tensor([[[1., 2.],
         [1., 2.],
         [1., 2.]],

        [[3., 4.],
         [3., 4.],
         [3., 4.]]])
torch.Size([2, 3, 2])


### randperm: Random Permutation

In [9]:
x = torch.randperm(10)

print(x)
print(x.size())

tensor([9, 1, 3, 4, 5, 0, 2, 7, 8, 6])
torch.Size([10])


### argmax: Return index of maximum values

In [10]:
x = torch.randperm(3**3).reshape(3, 3, -1)

print(x)
print(x.size())

tensor([[[18, 13,  5],
         [15, 16, 26],
         [17, 25,  3]],

        [[24, 21, 23],
         [ 8,  7, 14],
         [ 4, 19,  2]],

        [[22,  9,  0],
         [ 6, 20,  1],
         [11, 12, 10]]])
torch.Size([3, 3, 3])


In [12]:
y = x.argmax(dim=-1)
z = x.argmax(dim=1)

print(y)
print(y.size())
print('')
print(z)
print(z.size())

tensor([[0, 2, 1],
        [0, 2, 1],
        [0, 1, 1]])
torch.Size([3, 3])

tensor([[0, 2, 1],
        [0, 0, 0],
        [0, 1, 2]])
torch.Size([3, 3])


In [14]:
my = torch.randperm(30).reshape(5,3,2)c # 30개 random. 0~29 사이의 수 
print(my)

tensor([[[16, 15],
         [11,  9],
         [ 4,  6]],

        [[26,  0],
         [17, 28],
         [ 5, 20]],

        [[ 1, 12],
         [29, 25],
         [19,  3]],

        [[24, 14],
         [10,  8],
         [ 2, 23]],

        [[13, 18],
         [ 7, 27],
         [22, 21]]])


In [16]:
my_y = my.argmax(dim=-1)

print(my_y)
print(my_y.size())

tensor([[0, 0, 1],
        [0, 1, 1],
        [1, 0, 0],
        [0, 0, 1],
        [1, 1, 0]])
torch.Size([5, 3])


### topk: Return tuple of top-k values and indices.

In [17]:
values, indices = torch.topk(x, k=1, dim=-1) # value, index return

print(values.size()) # k=1이라서 3,3,1
print(indices.size()) # returnr값이 (3,3,k)로 나옴

# k=1을 뽑는 경우가 대다수 -> argmax를 사용한 것과 동일

torch.Size([3, 3, 1])
torch.Size([3, 3, 1])


Note that topk didn't reduce the dimension, even in $k=1$ case.

In [18]:
print(values.squeeze(-1))
print(indices.squeeze(-1))

tensor([[18, 26, 25],
        [24, 14, 19],
        [22, 20, 12]])
tensor([[0, 2, 1],
        [0, 2, 1],
        [0, 1, 1]])


In [19]:
print(x.argmax(dim=-1) == indices.squeeze(-1))

tensor([[True, True, True],
        [True, True, True],
        [True, True, True]])


In [11]:
_, indices = torch.topk(x, k=2, dim=-1)
print(indices.size())

print(x.argmax(dim=-1) == indices[:, :, 0]) # indices[:,:,0] = (3,3) 차원
# index range할 때만 해당 차원이 살아있고, index자체를 허용했기 때문에 해당 차원 날라감

torch.Size([3, 3, 2])
tensor([[True, True, True],
        [True, True, True],
        [True, True, True]])


### Sort by using topk

In [20]:
target_dim = -1
values, indices = torch.topk(x,
                             k=x.size(target_dim), # 3 -> 큰 거 순서대로 3개 뽑아옴 == sort
                             largest=True)

print(values)

tensor([[[18, 13,  5],
         [26, 16, 15],
         [25, 17,  3]],

        [[24, 23, 21],
         [14,  8,  7],
         [19,  4,  2]],

        [[22,  9,  0],
         [20,  6,  1],
         [12, 11, 10]]])


### Topk by using sort

In [21]:
k=1
values, indices = torch.sort(x, dim=-1, descending=True)
values, indices = values[:, :, :k], indices[:, :, :k]

print(values.squeeze(-1)) # matrix(3,3)
print(indices.squeeze(-1))

tensor([[18, 26, 25],
        [24, 14, 19],
        [22, 20, 12]])
tensor([[0, 2, 1],
        [0, 2, 1],
        [0, 1, 1]])


**CPU에서는 Topk가 빠르고, GPU에서는 sort가 빠르고..**

### masked_fill: fill the value if element of mask is True.

In [22]:
x = torch.FloatTensor([i for i in range(3**2)]).reshape(3, -1)

print(x)
print(x.size())

tensor([[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.]])
torch.Size([3, 3])


In [23]:
mask = x > 4

print(mask)

tensor([[False, False, False],
        [False, False,  True],
        [ True,  True,  True]])


In [24]:
y = x.masked_fill(mask, value=-1) # mask가 true면 -1로 채워라

print(y)

tensor([[ 0.,  1.,  2.],
        [ 3.,  4., -1.],
        [-1., -1., -1.]])


### Ones and Zeros

In [25]:
print(torch.ones(2, 3)) # 그냥 ones는 CPU에 생성됨
print(torch.zeros(2, 3))

tensor([[1., 1., 1.],
        [1., 1., 1.]])
tensor([[0., 0., 0.],
        [0., 0., 0.]])


In [18]:
x = torch.FloatTensor([[1, 2, 3],
                       [4, 5, 6]])
print(x.size())

torch.Size([2, 3])


x의 디바이스 확인.. 변환.. (번거로움)

연산을 하기 위해서는 데이터 타입, 디바이스(CPU, GPU)도 같아야함
- CPU에 있는 것과 GPU에 있는 것을 연산시키면 죽음(에러)

In [30]:
check = torch.ones_like(x)
print(torch.ones_like(x)) # 타입, 디바이스 같음
print(torch.zeros_like(x))

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])


In [38]:
print(type(check))

<class 'torch.Tensor'>
